In [18]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim # 最佳化資料庫
from torchvision import datasets
from matplotlib import pyplot as plt
from torchvision import transforms

為tensor

In [19]:
import os
data_path = os.getcwd() # 在datasets有內建
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transforms.ToTensor()) # 第一次下載download為True 
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transforms.ToTensor()) # val不用train

Files already downloaded and verified
Files already downloaded and verified


標準化

In [20]:
# 變四軸(數據較大)
#imgs = torch.stack([img_t for img_t, _ in cifar10], dim=3) # 把tensor_cifar10中所有的5000張圖片放在第四維
# img_t.shape為共5000張torch.Size([3, 32, 32])的圖片疊加
#imgs.shape, 3*32*32

In [21]:
# 1列 其他自動合併(32*32*50000) 為橫向相加求平均
#imgs.view(3, -1).mean(dim=1), imgs.view(3, -1).std(dim=1)

為tensor和transform

In [22]:
# 訓練集
transformed_cifar10 = datasets.CIFAR10(
  data_path, train=True, download=True,
  transform=transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465),
  (0.2470, 0.2435, 0.2616))
]))

# val組
transformed_cifar10_val = datasets.CIFAR10(
  data_path, train=False, download=True,
  transform=transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465),
  (0.2470, 0.2435, 0.2616))
]))

Files already downloaded and verified
Files already downloaded and verified


分辨飛機與鳥cifar2

In [23]:
# 只取鳥和飛機
label_map = {0: 0, 2: 1} # {飛機: 新的array的0位置, 鳥: 新的array的1位置} ex.為0就轉換為0 為2就轉換為1
class_names = ['airplane', 'bird'] 

# 若label為0和2 匯出其img和label label放到label_map轉換整數值
cifar2 = [(img, label_map[label]) 
      for img, label in transformed_cifar10
      if label in [0, 2]]
cifar2_val = [(img, label_map[label])
        for img, label in transformed_cifar10_val
        if label in [0, 2]]

DataLoader

In [24]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=16, shuffle=True) # DataLaoder去批次(一次64個數據)訓練
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=16, shuffle=False) # 不用洗牌

訓練模型(LogSoftmax)

In [25]:
# 訓練層
"""
model = nn.Sequential(
      nn.Linear(3072, 512),
      nn.Tanh(),
      nn.Linear(512, 2),
      nn.LogSoftmax(dim=1))
"""

'\nmodel = nn.Sequential(\n      nn.Linear(3072, 512),\n      nn.Tanh(),\n      nn.Linear(512, 2),\n      nn.LogSoftmax(dim=1))\n'

In [26]:
model = nn.Sequential(
      nn.Linear(3072, 1024),
      nn.Tanh(),
      nn.Linear(1024, 512),
      nn.Tanh(),
      nn.Linear(512, 128),
      nn.Tanh(),
      nn.Linear(128, 2),
      nn.LogSoftmax(dim=1))

In [27]:
loss_fn = nn.NLLLoss() # 其相對應的loss function

In [28]:
learning_rate = 1e-2 # 學習率
optimizer = optim.SGD(model.parameters(), lr=learning_rate) # 優化器

In [29]:
n_epochs = 20
for epoch in range(n_epochs):
  for imgs, labels in train_loader: # 已經只有鳥和飛機的資料庫 且設定批次訓練64個 train_loader
    batch_size = imgs.shape[0] # 一次訓練16個 原始的為[64,16,4] 取到的為[1,16,4]

    outputs = model(imgs.view(batch_size, -1)) # [16, 3072]
    loss = loss_fn(outputs, labels) # outputs為訓練出來model猜的機率 torch.tensor([label])為label_map[label]為真正的機率0或1 訓練再回饋

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.523358
Epoch: 1, Loss: 0.247687
Epoch: 2, Loss: 0.297365
Epoch: 3, Loss: 0.300489
Epoch: 4, Loss: 0.445316
Epoch: 5, Loss: 0.494037
Epoch: 6, Loss: 0.741359
Epoch: 7, Loss: 0.329826
Epoch: 8, Loss: 0.216138
Epoch: 9, Loss: 0.072597
Epoch: 10, Loss: 0.303872
Epoch: 11, Loss: 0.252371
Epoch: 12, Loss: 0.546081
Epoch: 13, Loss: 0.494590
Epoch: 14, Loss: 0.192157
Epoch: 15, Loss: 0.040623
Epoch: 16, Loss: 0.140314
Epoch: 17, Loss: 0.087390
Epoch: 18, Loss: 0.738259
Epoch: 19, Loss: 0.115851


In [30]:
for imgss, labelss in train_loader: # 已經只有鳥和飛機的資料庫 且設定批次訓練64個 train_loader
  batch_size = imgss.shape[0]
imgss.shape[0]
imgss.view(batch_size, -1).shape

torch.Size([16, 3072])

正確率

In [31]:
correct = 0 
total = 0

with torch.no_grad(): # 不用回傳
  for imgs, labels in val_loader: # 驗證集
    
    batch_size = imgs.shape[0]
    outputs = model(imgs.view(batch_size, -1)) # 驗證集訓練結果

    _, predicted = torch.max(outputs, dim=1) # 回傳 (最大值的數字, 最大值的label) 在此為驗證的label

    total += labels.shape[0] # 每次訓練幾筆 都疊加 得到總共訓練的筆數
    correct += int((predicted == labels).sum()) # 若預測的結果和labels相似 則相加 得到總共的正確筆數

print("Accuracy: %f", correct / total)

Accuracy: %f 0.8235


去看主要是哪層hidden_layer貢獻w和b

In [32]:
numel_list = [p.numel() # 回傳數量
        for p in model.parameters() 
        if p.requires_grad == True] # 有貢獻到回傳的
# Parameter有兩個參數(data=None, requires_grad=True)
sum(numel_list), numel_list # 顯示1024貢獻了大多數的訓練 所以可以只用一層即可(效率更好)

(3737474, [3145728, 1024, 524288, 512, 65536, 128, 256, 2])